<a href="https://colab.research.google.com/github/RohaArslan/COMPUTERVISION/blob/main/harris_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import numpy as np

# Given image matrix
image = np.array([
    [20, 30, 10, 25, 80],
    [ 1,  1,  1,  1,  6],
    [10, 10, 10, 10, 10],
    [25, 10, 10, 10, 10],
    [50, 10, 10, 10, 10]
], dtype=float)

# Sobel kernels
K_x = np.array([[-1, 0, 1],
                [-2, 0, 2],
                [-1, 0, 1]])

K_y = np.array([[ 1,  2,  1],
                [ 0,  0,  0],
                [-1, -2, -1]])

# Function to apply convolution
def convolve2d(image, kernel):
    kernel_height, kernel_width = kernel.shape
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2
    padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='edge')
    output = np.zeros_like(image)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded_image[i:i + kernel_height, j:j + kernel_width]
            output[i, j] = np.sum(region * kernel)

    return output

# Calculate gradients
I_x = convolve2d(image, K_x)
I_y = convolve2d(image, K_y)

print("I_x (x-derivative):")
print(I_x)
print("\nI_y (y-derivative):")
print(I_y)
# Compute products of derivatives
I_xx = I_x ** 2  # Square of x-derivative
I_yy = I_y ** 2  # Square of y-derivative
I_xy = I_x * I_y # Product of x and y derivatives
print("\nI_xx (x-derivative squared):")
print(I_xx)
print("\nI_yy (y-derivative squared):")
print(I_yy)
print("\nI_xy (product of derivatives):")
print(I_xy)


I_x (x-derivative):
[[  30.  -30.  -15.  215.  170.]
 [  10.  -10.   -5.   80.   65.]
 [ -15.  -15.    0.    5.    5.]
 [ -70.  -70.    0.    0.    0.]
 [-135. -135.    0.    0.    0.]]

I_y (y-derivative):
[[  86.   86.   71.  131.  246.]
 [  50.   50.   35.  100.  225.]
 [ -81.  -51.  -36.  -31.  -21.]
 [-120.  -40.    0.    0.    0.]
 [ -75.  -25.    0.    0.    0.]]

I_xx (x-derivative squared):
[[9.0000e+02 9.0000e+02 2.2500e+02 4.6225e+04 2.8900e+04]
 [1.0000e+02 1.0000e+02 2.5000e+01 6.4000e+03 4.2250e+03]
 [2.2500e+02 2.2500e+02 0.0000e+00 2.5000e+01 2.5000e+01]
 [4.9000e+03 4.9000e+03 0.0000e+00 0.0000e+00 0.0000e+00]
 [1.8225e+04 1.8225e+04 0.0000e+00 0.0000e+00 0.0000e+00]]

I_yy (y-derivative squared):
[[ 7396.  7396.  5041. 17161. 60516.]
 [ 2500.  2500.  1225. 10000. 50625.]
 [ 6561.  2601.  1296.   961.   441.]
 [14400.  1600.     0.     0.     0.]
 [ 5625.   625.     0.     0.     0.]]

I_xy (product of derivatives):
[[ 2580. -2580. -1065. 28165. 41820.]
 [  500.  -500.

In [45]:
# Harris response calculation
def harris_response(I_xx, I_yy, I_xy, k=0.04):
    height, width = I_xx.shape
    R = np.zeros((height, width))

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            # Compute the structure tensor M
            M = np.array([[np.sum(I_xx[i-1:i+2, j-1:j+2]), np.sum(I_xy[i-1:i+2, j-1:j+2])],
                          [np.sum(I_xy[i-1:i+2, j-1:j+2]), np.sum(I_yy[i-1:i+2, j-1:j+2])]])

            # Calculate R
            det_M = np.linalg.det(M)
            trace_M = np.trace(M)
            R[i, j] = det_M - k * (trace_M ** 2)

    return R

# Compute Harris response R
R = harris_response(I_xx, I_yy, I_xy)

# Function to classify pixels
def classify_pixels(R):
    classifications = np.zeros_like(R, dtype=str)
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if R[i, j] > 0:  # Corner
                classifications[i, j] = "corner pixel"
            elif R[i, j] < 0:  # Edge
                classifications[i, j] = "edge pixel"
            else:  # Flat region
                classifications[i, j] = "flat region pixel"
    return classifications

# Classify pixels
classifications = classify_pixels(R)

# Display results for each pixel
print("\nHarris Response R:")
print(R)

print("\nClassifications:")
for i in range(R.shape[0]):
    for j in range(R.shape[1]):
        print(f"Pixel ({i}, {j}): R = {R[i, j]:.4f}, Classification: {classifications[i, j]}")


Harris Response R:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  3.65298138e+07  1.13580889e+09  2.19375297e+09
   0.00000000e+00]
 [ 0.00000000e+00  9.87198814e+07  7.97990134e+07 -2.82229602e+07
   0.00000000e+00]
 [ 0.00000000e+00  5.63415981e+08  8.24213094e+07 -2.34760160e+05
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]

Classifications:
Pixel (0, 0): R = 0.0000, Classification: f
Pixel (0, 1): R = 0.0000, Classification: f
Pixel (0, 2): R = 0.0000, Classification: f
Pixel (0, 3): R = 0.0000, Classification: f
Pixel (0, 4): R = 0.0000, Classification: f
Pixel (1, 0): R = 0.0000, Classification: f
Pixel (1, 1): R = 36529813.7600, Classification: c
Pixel (1, 2): R = 1135808894.5600, Classification: c
Pixel (1, 3): R = 2193752965.7600, Classification: c
Pixel (1, 4): R = 0.0000, Classification: f
Pixel (2, 0): R = 0.0000, Classification: f
Pixel (2, 1): R = 98719881.